In [12]:

%load_ext autoreload 
# %aimport rl_envs.grid_world_env

%autoreload 2
import math

from agents.approx_Q_Learning import ApproxQLearningAgent
from tools.helper import *
import  gymnasium  as gym
from rl_envs.new_gym_grid_world_env import GridWorldEnv
from collections import defaultdict
from torch.utils.tensorboard import SummaryWriter # type: ignore
import itertools
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
DISCOUNTED_FACTOR = 0.999
FORBIDDEN_REWARD = -10
HITWALL_REWARD = -10
TARGET_REWARD = 15
NORMAL_REWARD = -1

SEED = 666

In [14]:
# env = GridWorldEnv(
#     size=3,
#     fixed_map=True,
#     forbidden_grids=[(1, 1), (1, 2)],
#     target_grids=[(2, 2)],
#     forbidden_reward=FORBIDDEN_REWARD,
#     hit_wall_reward=HITWALL_REWARD,
#     target_reward=TARGET_REWARD,
#     normal_reward=NORMAL_REWARD,
# )
env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD, normal_reward=NORMAL_REWARD)


In [15]:
agent = ApproxQLearningAgent(action_space_n=env.action_n)

agent.RUN(env, num_episodes=300000, episode_len=20000)
# agent.RUN(env, num_episodes=10000, episode_len=2000)
# num_episodes=1000, episode_len=2000 能解出 3x3

Episode 0/300000 ( TD_error: -11.080687917718185, reward: -14.0,0.0)
Episode 100/300000 ( TD_error: -10.019152515903361, reward: -10.0,-28.0)
Episode 200/300000 ( TD_error: -10.170356538652534, reward: -10.0,-27.0)
Episode 300/300000 ( TD_error: -10.132695912817876, reward: -10.0,-24.0)
Episode 400/300000 ( TD_error: -3.743118625424478, reward: -55.0,-10.0)
Episode 500/300000 ( TD_error: -10.10662452094772, reward: -10.0,5.0)
Episode 600/300000 ( TD_error: -7.176641704933494, reward: -18.0,-36.0)
Episode 700/300000 ( TD_error: -2.953028389241632, reward: -11.0,-24.0)
Episode 800/300000 ( TD_error: 1.075519145906107, reward: -99.0,-13.0)
Episode 900/300000 ( TD_error: -5.5540859114656325, reward: -10.0,-12.0)
Episode 1000/300000 ( TD_error: -0.917047412038851, reward: -25.0,-18.0)
Episode 1100/300000 ( TD_error: -2.3055723134172, reward: -74.0,-13.0)
Episode 1200/300000 ( TD_error: 6.9962024513018415, reward: 15.0,-10.0)
Episode 1300/300000 ( TD_error: -3.108541950922315, reward: -50.0,

In [16]:
for y in range(env.height):
    for x in range(env.width):
        agent.policy_improvement((y, x))

Q = defaultdict(lambda: [0 for _ in range(agent.action_space_n)])
for y in range(env.height):
    for x in range(env.width):
        for action in range(agent.action_space_n):
            state = (y,x)
            Q[state][action] = agent.estimate_q(state, action)
print_by_dict(env, Q)

# ISSUE: 在 X 相同时, 不同的 y, q 值保持一致 (为何)


[ [-6.986939871948687, -6.017987442664804, -12.82130229356874, -15.729371773779, -8.144584422024948] [-14.948476303672813, -5.435374676388051, -15.046405535585489, -6.820336250706031, -5.893941972127547] [-13.010697800374354, -4.895237975595911, -13.458164004250266, -7.682371150952827, -7.060598082205252] [-4.376187793479181, -4.978496690306164, -13.44546824929856, -5.358084965153643, -5.625881222719211] [-5.940826814161074, -13.499279334034895, -10.158272267571544, -4.6915123850517855, -8.736422456323695] ]
[ [-10.871187747831273, -14.717985181560973, -6.619929242410411, -15.829675754589314, -7.972361619192998] [-7.870913434087603, -10.468485332732818, -5.559513287558136, -7.107411645213768, -10.46519223393127] [-7.283500686032866, -3.788601511111757, -3.994608392552353, -18.820609890696183, -14.56444103242526] [-5.538732867013346, -3.796469289323062, -4.476612544271437, -12.669542554741229, -4.556986414494121] [-3.190148796275834, -12.689474754339427, -4.703767319849864, -3.988778274

In [18]:
print_by_dict(env, agent.policy)
print()
print(agent.parameters)

print(env)
for i in range(env.height):
    print("[", end=" ")
    for j in range(env.width):
        state = (i, j)
        action = agent.get_action(state, optimal=True)
        print(env.action_mappings[action], end=" ")
    print("]")

[ [0.02 0.92 0.02 0.02 0.02] [0.02 0.92 0.02 0.02 0.02] [0.02 0.92 0.02 0.02 0.02] [0.92 0.02 0.02 0.02 0.02] [0.02 0.02 0.02 0.92 0.02] ]
[ [0.02 0.02 0.92 0.02 0.02] [0.02 0.02 0.92 0.02 0.02] [0.02 0.92 0.02 0.02 0.02] [0.02 0.92 0.02 0.02 0.02] [0.92 0.02 0.02 0.02 0.02] ]
[ [0.02 0.02 0.92 0.02 0.02] [0.02 0.92 0.02 0.02 0.02] [0.02 0.02 0.02 0.02 0.92] [0.02 0.02 0.02 0.02 0.92] [0.92 0.02 0.02 0.02 0.02] ]
[ [0.92 0.02 0.02 0.02 0.02] [0.02 0.92 0.02 0.02 0.02] [0.92 0.02 0.02 0.02 0.02] [0.02 0.92 0.02 0.02 0.02] [0.92 0.02 0.02 0.02 0.02] ]
[ [0.02 0.02 0.02 0.02 0.92] [0.02 0.02 0.02 0.02 0.92] [0.02 0.02 0.92 0.02 0.02] [0.02 0.02 0.02 0.92 0.02] [0.02 0.02 0.02 0.92 0.02] ]

[-8.05089112e+00  8.71125293e-01 -1.90221208e-01 -2.45791418e-01
 -1.52533256e+00 -1.92655068e+00  6.36564159e-01 -5.50471144e-01
 -3.06418769e-01 -5.58078027e-01  9.33336930e-01  7.94355503e-01
  1.17180668e+00  9.12374136e-01  2.71454508e-01 -9.16038572e-01
 -8.59285976e-01  3.11263543e-01  2.14833366